In [112]:
# 這邊放要import的套件
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time


In [150]:
# 1. 設定變數
products = ['Apple iPad Air',
            'Apple AirPods Pro 2',
            'Apple Adapter',
            'Apple Watch Series 10',
            'Apple TV 4K']

retailer_web = {
    # 供應商 : 網站首頁連結
    'Amazon': 'https://www.amazon.com',
    'Best Buy': 'https://www.bestbuy.com',
    'Target': 'https://www.target.com',
    'Costco': 'https://www.costco.com',
    # 'Newegg':'https://www.newegg.com',
    'BJ': 'https://www.bjs.com',
}

In [152]:
from selenium.common.exceptions import ElementClickInterceptedException

def search_amazon(driver, product):
    # Check if there is a check page
    try:
        captcha_text = driver.find_element(By.XPATH, "//h4[contains(text(), 'Enter the characters you see below')]")
        if captcha_text:
            input("Press Enter to continue...")    
        time.sleep(3)
    except Exception:
        pass
    
    # Find the search bar element
    search_box = driver.find_element(By.ID, 'twotabsearchtextbox')
    search_box.clear()
    search_box.send_keys(product)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)

    # Click product
    product_link = driver.find_element(By.CSS_SELECTOR, 'a.a-link-normal.s-no-outline')
    product_link.click()
    time.sleep(3)

    # Collect product prices
    price_element = driver.find_element(By.CLASS_NAME, 'a-price-whole')
    price = price_element.text if price_element else 0
   
    # Collect product review
    review_div = driver.find_element(By.ID, 'averageCustomerReviews')
    review_element = review_div.find_element(By.CLASS_NAME, "a-size-base.a-color-base")
    review = review_element.text if review_element else ''

    return price, review

def search_bestbuy(driver, product):
    search_box = driver.find_element(By.CSS_SELECTOR, 'input.search-input')  # Adjust selector
    search_box.clear()
    search_box.send_keys(product)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)  

    # Click product
    product_link = driver.find_element(By.CLASS_NAME, 'image-link')
    product_link.click()
    time.sleep(3)

    # Collect product prices
    price_div = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="customer-price"]')
    price_element = price_div.find_element(By.TAG_NAME, "span")
    price = price_element.text if price_element else 0
   
    # Collect product review
    review_element = driver.find_element(By.CSS_SELECTOR, 'span.ugc-c-review-average.font-weight-medium.order-1')
    review = review_element.text if review_element else ''

    return price, review

# def search_newegg(driver, product):
#     search_box = WebDriverWait(driver, 10).until(
#         EC.element_to_be_clickable((By.XPATH, '//input[@type="search" and @title="Search Site"]'))
#     )
#     search_box.clear()
#     search_box.send_keys(product)
#     search_box.send_keys(Keys.RETURN)
#     time.sleep(5)

#     price_element = WebDriverWait(driver, 10).until(
#         EC.visibility_of_element_located((By.CSS_SELECTOR, 'li.price-current'))
#     )
#     return price_element[0].text if price_element else 0

def search_target(driver, product):
    search_box = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'search'))
    )
    search_box.clear()
    search_box.send_keys(product)
    search_box.send_keys(Keys.RETURN)
    time.sleep(5) 
    
    # Click product
    for _ in range(5):  # 最多重試5次
        try:
            product_link = driver.find_element(By.CSS_SELECTOR, 'a[data-test="product-title"]')
            product_link.click()
            time.sleep(5) 
            break 
        except ElementClickInterceptedException:
            # 元素被遮擋，等待並重試
            time.sleep(2)  

    # Collect product prices
    price_element = driver.find_element(By.CSS_SELECTOR, 'span[data-test="product-price"]')
    price = price_element.text if price_element else 0

    # Collect product review
    review_element = driver.find_element(By.CSS_SELECTOR, 'span[data-test="ratings"] > span')
    full_review = review_element.text
    review = full_review.split(" ")[0] 

    return price, review

def search_costco(driver, product):
    search_box = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//input[@placeholder="Search Costco"]'))
    )
    search_box.clear()
    search_box.send_keys(product)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)
    
    # Click product
    product_list_div = driver.find_element(By.ID, "productList")
    product_link = product_list_div.find_element(By.CSS_SELECTOR, 'a[data-testid="Link"]')
    product_link.click()
    time.sleep(5)

    # Collect product prices
    price_element = driver.find_element(By.CSS_SELECTOR, 'span[automation-id="productPriceOutput"]')
    price = price_element.text if price_element else 0

    # Collect product review
    review_element = driver.find_element(By.CSS_SELECTOR, 'div[itemprop="ratingValue"]')
    review = review_element.text if review_element else ''

    return price, review

def search_bj(driver, product):
    # Close pop up
    try:
        close_button = driver.find_element(By.XPATH, '//button[@aria-label="Close"]')
        close_button.click()
    except:
        pass
    
    search_box = driver.find_element(By.CSS_SELECTOR, 'input[type="search"]')
    search_box.clear()
    search_box.send_keys(product)
    search_box.send_keys(Keys.RETURN)
    time.sleep(5)

    # Click product
    div_element = driver.find_element(By.CSS_SELECTOR, 'div.ProductImagestyle__ProductImageStyle-sc-1l5x5t5-0.keMjrr')
    product_link = div_element.find_element(By.TAG_NAME, "a")
    product_link.click()

    # Collect product prices
    price_div = driver.find_element(By.CSS_SELECTOR, 'div[auto-data="product_price"]')
    spans = price_div.find_elements(By.TAG_NAME, "span")
    price = spans[1].text if spans[1].text else 0

    # Collect product review
    try:
        rating_element = driver.find_element(By.CSS_SELECTOR, 'span[data-testid="review-count"]')
        actions = ActionChains(driver)
        actions.move_to_element(rating_element).perform()
        time.sleep(1)
        review_element = driver.find_element(By.CSS_SELECTOR, 'span.ReviewDropdown__StyledReviewRating-sc-6tlq5u-3.HMqma')
        review = review_element.text if review_element else ''

    except Exception:
        review = 'No review now'
    
    return price, review

In [154]:
def search_product(product, retailer, link):
    
    # Set up Selenium WebDriver (example with Chrome)
    driver = webdriver.Chrome()
    driver.get(link)

    # Give the page some time to load (adjust as needed)
    time.sleep(5)
    product_price = 0
    
    try:
        if retailer == 'Amazon':
            product_price = search_amazon(driver, product)

        elif retailer == 'Best Buy':
            product_price = search_bestbuy(driver, product)
                    
        elif retailer == 'Target':
            product_price = search_target(driver, product)

        elif retailer == 'Costco':
            product_price = search_costco(driver, product)
        
        elif retailer == 'BJ':
            product_price = search_bj(driver, product)

    except Exception as e:
        print("Search product fail: ", e)
    
    driver.quit()
    return product_price if product_price != 0 else "Product price not found"

# 主要程式碼
執行下面的會把所有商家跟產品跑過

In [157]:
retailer_prices = {}

# 遍歷五家供應商
for retailer, link in retailer_web.items():
    

    print(retailer)
    
    # 遍歷五個產品
    product_info = []
    for product in products:

        # 針對某個商家搜尋某個產品價的格
        # output = function(input)
        info = search_product(product, retailer, link)
        # 把價格存到 list
        product_info.append(info)
        print(product, ': ', info)

    # 把價格存回去
    retailer_prices[retailer] = product_info

Amazon
Apple iPad Air :  ('549', '4.7')
Search product fail:  Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="twotabsearchtextbox"]"}
  (Session info: chrome=130.0.6723.117); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001008915dc cxxbridge1$str$ptr + 3653648
1   chromedriver                        0x0000000100889e3c cxxbridge1$str$ptr + 3623024
2   chromedriver                        0x00000001002f4100 cxxbridge1$string$len + 88404
3   chromedriver                        0x000000010033641c cxxbridge1$string$len + 359536
4   chromedriver                        0x000000010036fba8 cxxbridge1$string$len + 594940
5   chromedriver                        0x000000010032b0fc cxxbridge1$string$len + 313680
6   chromedriver                        0x000000010032bd4c cxxbridge1$string$len + 3168

## 測試區

In [ ]:
func_options = {
    1: search_amazon,
    2: search_bestbuy,
    3: search_target,
    4: search_costco,
    5: search_bj,
}

link_options = {
    1: retailer_web['Amazon'],
    2: retailer_web['Best Buy'],
    3: retailer_web['Target'],
    4: retailer_web['Costco'],
    5: retailer_web['BJ'],
}

def get_product_info(product):
    print(product)

    link = link_options.get(option)
    driver = webdriver.Chrome()
    driver.get(link)
    
    price, review = func_options.get(option, lambda: "Invalid option")(driver, product)
    print('price:', price, ', review: ', review)

# 設定要不要跑過所有產品
all = 1
# 設定哪一間廠商
option = 6
# 除果 all=0 的話要跑的單一產品名稱
product = 'Apple AirPods Pro'
    
try:
    if all == 1: 
        for product in products:
            get_product_info(product)
    else:
        get_product_info(product)

finally:
    print('end')

# 關閉瀏覽器
# driver.quit()

Apple iPad Air
price: 569.99 , review:  
Apple AirPods Pro
price: 189.99 , review:  5
Apple Adapter
price: 17.99 , review:  5
Dell Inspiron Laptop
price: 399.99 , review:  No review now
Asus ZenBook 14
price: 1079.99 , review:  No review now
end
